In [ ]:
import Pkg
#Pkg.add("MathOptInterface")
#Pkg.add("GLPK")
Pkg.activate(@__DIR__)
Pkg.instantiate()

In [ ]:
using JuMP
using MathOptInterface # Replaces MathProgBase
# shortcuts
const MOI = MathOptInterface
const MOIU = MathOptInterface.Utilities

using GLPK # Loading the GLPK module for using its solver

In [ ]:
include("../src/read_data.jl")
include("../JuMP/main_JuMP.jl")

In [ ]:
JJ=[j for j in 1:J]
PP=[p for p in 1:P]
CC=[c for c in 1:C]
TT=[t for t in 0:H-1]
QQ=[q for q in 1:Q]

In [ ]:
model = Model(with_optimizer(GLPK.Optimizer, msg_lev = 4));  
# Old syntax: model = Model(solver=GLPKSolverLP(msg_lev = 4)))

In [ ]:
@variable(model, x[p=1:P, i=1:C, q=1:Q], Bin)
@variable(model, y[j=1:J, q=1:Q, t=0:H-1], Bin)
@variable(model, w[j=1:J, q=1:Q, t=0:H-1], Bin)
@variable(model, flag[t=0:H-1], Bin)

@variable(model, t_load[p=1:P, q=1:Q], lower_bound=0, upper_bound=H)


#objective function
@objective(model, Min, -sum(flag[t] for t in TT))

In [ ]:
@constraint(model, [i=1:C], sum(sum(x[p,i,q] for q in subset_crane_pos(CTS, p, bj)) for p in subset_pos(PP, tasks_by_position, i)) == 1)
@constraint(model, [p=1:P], sum(sum(x[p,i,q] for q in subset_crane_pos(CTS, p, bj)) for i in subset_pos(CC, tasks_by_position, p)) == 1)

In [ ]:
@constraint(model, [t=0:H-1, q=1:Q], sum(y[j,q,t] for j in JJ) == 1)
@constraint(model, [t=0:H-1, j=1:J], sum(y[j,q,t] for q in QQ) <= 1)

In [ ]:
for i = 1:C
    for p = 1:P
        for q in setdiff(Set(QQ),Set(subset_crane_pos(CTS, p, bj)))
            @constraint(model, x[p,i,q] == 0)
        end
    end
end

for t = 0:H-1
    for q = 1:Q
        for j in setdiff(Set(JJ),Set(subset_bay(CTS, q)))
            @constraint(model, y[j,q,t] == 0)
        end
    end
end

In [ ]:
@constraint(model, [t=0:H-1, q=1:Q, s=1:Q, j=2:J], 1-y[j,q,t] >= sum(y[l,s,t] for l in collect(max(1,j-d):j-1)))
@constraint(model, [t=0:H-1, q=1:Q, s=1:Q, j=1:J-1], 1-y[j,q,t] >= sum(y[l,s,t] for l in collect(j+1:min(j+d,J))))
@constraint(model, [t=0:H-1, q=1:Q-1, j=1:J-1], y[j,q,t] <= sum(y[l,q+1,t] for l in collect(j+1:J)))
@constraint(model, [t=0:H-1, q=2:Q, j=2:J], y[j,q,t] <= sum(y[l,q-1,t] for l in collect(1:j-1)))

In [ ]:
for t = 0:H-1
    for q = 1:Q
        for j in subset_bay(CTS, q)
            @constraint(model, w[j,q,t] <= y[j,q,t])
        end
    end
end

for t = 0:H-1
    for p = 1:P
        for q in subset_crane_pos(CTS, p, bj)
            @constraint(model, sum(x[p,i,q] for i in subset_pos(CC, tasks_by_position, p)) <= w[bj[p],q,t])
        end
    end
end

for p = 1:P
    for pp in prec[p]
        @constraint(model, sum(t_load[p,q] for q in subset_crane_pos(CTS, p, bj)) - sum(t_load[pp,q] for q in subset_crane_pos(CTS, pp, bj)) >= 2*sum(sum(task_times[p,i]*x[p,i,q] for i in subset_pos(CC, tasks_by_position, p) for q in subset_crane_pos(CTS, p ,bj))))
    end
    @constraint(model, sum(t_load[p,q] for q in subset_crane_pos(CTS, p, bj)) >= 2*sum(sum(task_times[p,i]*x[p,i,q] for i in subset_pos(CC, tasks_by_position, p) for q in subset_crane_pos(CTS, p ,bj))))
end    

In [ ]:
@constraint(model, [t=0:H-1], flag[t] <= (sum(sum(sum(w[j,q,l] for q in QQ) for j in JJ) for l in collect(1:t)))/CTS.P)

## Solution

In [ ]:
JuMP.optimize!(model) # Old syntax: status = JuMP.solve(model)

In [ ]:
sol_x = Dict{Int, Array}()
for p=1:P
    for i=1:C
        for q=1:Q
            if JuMP.value.(x)[p,i,q] == 1
                if haskey(sol_x, q) == false
                    sol_x[q] = Array{NamedTuple{(:pos, :cont),Tuple{Int64,Int64}}, 1}()
                end
                push!(sol_x[q], (pos=p, cont=i))
            end                
        end
    end
end

sol_x[1]

In [ ]:
sol_x[2]

In [ ]:
sol_t = JuMP.value.(t_load)

In [ ]:
sol_y = Dict{Int, Array}()
for t=0:H-1
    for j=1:J
        for q=1:Q
            if JuMP.value.(y)[j,q,t] == 1
                if haskey(sol_y, q) == false
                    sol_y[q] = Array{NamedTuple{(:time, :bay),Tuple{Int64,Int64}}, 1}()
                end
                push!(sol_y[q], (time=t, bay=j))
            end                
        end
    end
end

sol_y[1]

In [ ]:
sol_y[2]

In [ ]:
flag = JuMP.value.(flag)